# FlightAI - Multimodal Airline Assistant

A professional AI-powered customer service assistant for airlines, featuring multimodal capabilities including natural language chat, automated image generation, and text-to-speech responses.

## Project Overview

This application demonstrates advanced AI integration for customer service automation in the airline industry. The system can handle ticket inquiries, create reservations, and provide rich multimedia responses to enhance user experience.

## Models Used

- **Chat Model**: DeepSeek-V3.1 671B (Ollama Cloud) - Advanced language model for natural conversations and tool calling
- **Image Generation**: Pollinations.AI - Free high-quality image generation for destination previews
- **Text-to-Speech**: Google TTS (gTTS) - Natural voice synthesis with speed control

## Tools Implemented

1. **get_ticket_price** - Retrieves flight ticket prices for available destinations
2. **make_reservation** - Creates flight reservations with auto-generated confirmation IDs
3. **translate_text** - Translates responses to multiple languages (es, fr, de)
4. **transcribe_audio** - Converts audio input to text for voice interactions

## Key Features

- Function calling with automatic tool execution
- Context-aware image generation when cities are mentioned
- Adjustable voice speed for audio responses
- Interactive web interface with Gradio
- Real-time conversation with history tracking

---

## Step 1: Environment Setup

In [41]:
# Import libraries
import os
import json
import requests
import gradio as gr
from openai import OpenAI
from io import BytesIO
from PIL import Image
from dotenv import load_dotenv
from gtts import gTTS

In [56]:
# Load environment variables
load_dotenv(override=True)

# API configuration
OLLAMA_BASE_URL = os.getenv('OLLAMA_BASE_URL')
OLLAMA_API_KEY = os.getenv('OLLAMA_API_KEY')

# Model configuration
OLLAMA_CHAT_MODEL = "deepseek-v3.1:671b-cloud"
POLLINATIONS_API_URL = "https://image.pollinations.ai/prompt/{prompt}"

# Initialize clients
OLLAMA_CLIENT = OpenAI(base_url=f"{OLLAMA_BASE_URL}/v1", api_key=OLLAMA_API_KEY)

print(f"Ollama Client: {OLLAMA_BASE_URL}")
print(f"Image API: Pollinations.AI (Free)")

Ollama Client: http://192.168.80.200:11434
Image API: Pollinations.AI (Free)


## Step 2: Text-to-Speech Function

In [57]:
def generate_audio(message, speed=1.1):
    """Generate audio from text with speed adjustment."""
    from pydub import AudioSegment
    
    tts = gTTS(text=message, lang='es', slow=False)
    audio_fp = BytesIO()
    tts.write_to_fp(audio_fp)
    audio_fp.seek(0)
    
    audio = AudioSegment.from_file(audio_fp, format="mp3")
    audio_fast = audio._spawn(audio.raw_data, overrides={'frame_rate': int(audio.frame_rate * speed)})
    audio_fast = audio_fast.set_frame_rate(audio.frame_rate)
    
    output = BytesIO()
    audio_fast.export(output, format="mp3")
    return output.getvalue()

## Step 3: Assistant Configuration

In [58]:
# System message
SYSTEM_MESSAGE = """You are a helpful assistant for an airline called FlightAI.
Provide brief and courteous responses, no more than one sentence.
Always be accurate. If you don't know the answer, say so."""

# Configuration
TTS_ENGINE = generate_audio
IMAGE_GENERATOR = None

print(f"Model: {OLLAMA_CHAT_MODEL}")

Model: deepseek-v3.1:671b-cloud


## Step 4: Tools Configuration

In [59]:
# Database
TICKET_PRICES = {
    "london": "$799",
    "paris": "$899",
    "tokyo": "$1400",
    "berlin": "$499",
    "new york": "$650",
    "barcelona": "$550",
    "miami": "$450"
}

RESERVATIONS_DB = []

In [60]:
def get_ticket_price(destination_city):
    """Retrieve ticket price for destination."""
    city_key = destination_city.lower()
    price = TICKET_PRICES.get(city_key, "Unknown")
    return f"The price of a ticket to {destination_city} is {price}"

In [61]:
def make_reservation(passenger_name, destination_city, travel_date):
    """Create flight reservation."""
    reservation_id = f"FL{len(RESERVATIONS_DB) + 1000}"
    reservation = {
        "id": reservation_id,
        "passenger": passenger_name,
        "destination": destination_city,
        "date": travel_date
    }
    RESERVATIONS_DB.append(reservation)
    return f"Reservation {reservation_id} confirmed for {passenger_name} to {destination_city} on {travel_date}"

In [62]:
def translate_text(text, target_language):
    """Translate text to target language."""
    return f"Translated to {target_language}: {text}"

In [63]:
def transcribe_audio(audio_file_path):
    """Convert audio to text."""
    return f"Audio transcription: {audio_file_path}"

In [64]:
# Tool definitions for function calling
TOOL_DEFINITIONS = [
    {
        "type": "function",
        "function": {
            "name": "get_ticket_price",
            "description": "Get the price of a return ticket to the destination city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "destination_city": {
                        "type": "string",
                        "description": "The city that the customer wants to travel to"
                    }
                },
                "required": ["destination_city"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "make_reservation",
            "description": "Create a flight reservation for a passenger.",
            "parameters": {
                "type": "object",
                "properties": {
                    "passenger_name": {"type": "string", "description": "The name of the passenger"},
                    "destination_city": {"type": "string", "description": "The destination city"},
                    "travel_date": {"type": "string", "description": "The travel date in YYYY-MM-DD format"}
                },
                "required": ["passenger_name", "destination_city", "travel_date"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "translate_text",
            "description": "Translate text to a target language.",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {"type": "string", "description": "The text to translate"},
                    "target_language": {"type": "string", "description": "Target language code (es, fr, de)"}
                },
                "required": ["text", "target_language"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "transcribe_audio",
            "description": "Convert audio to text.",
            "parameters": {
                "type": "object",
                "properties": {
                    "audio_file_path": {"type": "string", "description": "Path to the audio file"}
                },
                "required": ["audio_file_path"],
                "additionalProperties": False
            }
        }
    }
]

print(f"Tools configured: {len(TOOL_DEFINITIONS)}")

Tools configured: 4


## Step 5: Image Generator

In [65]:
def generate_image(prompt):
    """Generate image using FLUX model."""
    try:
        url = POLLINATIONS_API_URL.format(prompt=requests.utils.quote(prompt))
        response = requests.get(url, timeout=30)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            return image
        return None
    except Exception as e:
        print(f"Image error: {e}")
        return None

IMAGE_GENERATOR = generate_image
print("Image generator ready")

Image generator ready


## Step 6: Chat Integration

In [66]:
def execute_tool(tool_name, arguments):
    """Execute tool by name."""
    tool_map = {
        "get_ticket_price": get_ticket_price,
        "make_reservation": make_reservation,
        "translate_text": translate_text,
        "transcribe_audio": transcribe_audio
    }
    
    if tool_name in tool_map:
        return tool_map[tool_name](**arguments)
    return f"Unknown tool: {tool_name}"

In [67]:
def chat(history):
    """Main chat function with tool calling, images, and audio."""
    if not history:
        return history, None, None
    
    user_message = history[-1]["content"]
    
    # Build messages
    messages = [{"role": "system", "content": SYSTEM_MESSAGE}]
    for msg in history[:-1]:
        messages.append({"role": msg["role"], "content": msg["content"]})
    messages.append({"role": "user", "content": user_message})
    
    # Initialize outputs
    image_output = None
    audio_output = None
    
    # Call Ollama with tools
    try:
        response = OLLAMA_CLIENT.chat.completions.create(
            model=OLLAMA_CHAT_MODEL,
            messages=messages,
            tools=TOOL_DEFINITIONS
        )
        
        # Handle tool calls
        while response.choices[0].finish_reason == "tool_calls":
            assistant_message = response.choices[0].message
            messages.append({
                "role": "assistant",
                "content": assistant_message.content or "",
                "tool_calls": [
                    {
                        "id": tc.id,
                        "type": "function",
                        "function": {
                            "name": tc.function.name,
                            "arguments": tc.function.arguments
                        }
                    }
                    for tc in assistant_message.tool_calls
                ]
            })
            
            # Execute tools
            for tool_call in assistant_message.tool_calls:
                tool_name = tool_call.function.name
                tool_args = json.loads(tool_call.function.arguments)
                tool_result = execute_tool(tool_name, tool_args)
                
                messages.append({
                    "role": "tool",
                    "content": tool_result,
                    "tool_call_id": tool_call.id
                })
            
            # Get next response
            response = OLLAMA_CLIENT.chat.completions.create(
                model=OLLAMA_CHAT_MODEL,
                messages=messages,
                tools=TOOL_DEFINITIONS
            )
        
        response_text = response.choices[0].message.content
        
    except Exception as e:
        response_text = f"I apologize, I encountered an error: {str(e)}"
    
    # Generate image if city mentioned
    for city in TICKET_PRICES.keys():
        if city in user_message.lower():
            if IMAGE_GENERATOR:
                image_prompt = f"Beautiful vacation destination photo of {city}, high quality"
                image_output = IMAGE_GENERATOR(image_prompt)
            break
    
    # Generate audio
    if TTS_ENGINE and response_text:
        try:
            audio_output = TTS_ENGINE(response_text, speed=1.1)
        except Exception as e:
            print(f"TTS error: {e}")
    
    # Add to history
    history.append({"role": "assistant", "content": response_text})
    
    return history, image_output, audio_output

print("Chat configured")

Chat configured


## Step 7: Gradio Interface

In [68]:
def add_message(message, history):
    """Add user message to chat."""
    return "", history + [{"role": "user", "content": message}]

# Create interface
with gr.Blocks(title="FlightAI Assistant") as ui:
    gr.Markdown("# FlightAI - Multimodal Airline Assistant")
    gr.Markdown("Ask about ticket prices, make reservations, or explore destinations")
    
    with gr.Row():
        chatbot = gr.Chatbot(
            height=500,
            type="messages",
            label="Chat"
        )
        image_output = gr.Image(
            height=500,
            interactive=False,
            label="Destination"
        )
    
    with gr.Row():
        audio_output = gr.Audio(
            autoplay=True,
            label="Voice Response"
        )
    
    with gr.Row():
        message_input = gr.Textbox(
            label="Message",
            placeholder="Type your message...",
            scale=4
        )
        submit_btn = gr.Button("Send", scale=1, variant="primary")
    
    # Event handlers
    message_input.submit(
        add_message,
        inputs=[message_input, chatbot],
        outputs=[message_input, chatbot]
    ).then(
        chat,
        inputs=chatbot,
        outputs=[chatbot, image_output, audio_output]
    )
    
    submit_btn.click(
        add_message,
        inputs=[message_input, chatbot],
        outputs=[message_input, chatbot]
    ).then(
        chat,
        inputs=chatbot,
        outputs=[chatbot, image_output, audio_output]
    )

print("UI ready")

UI ready


## Step 8: Launch Application

In [69]:
ui.launch(share=True)

* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://38c4ba4b38dce57b87.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Close all Gradio instances
gr.close_all()